# ASTR 400B Research Assignment 3:
This code is a draft for what will be used for my final project.
The general topic of my research assignment is analyzing how galaxies and dark matter halos evolve together through mergers. The major question is whether the dark matter halo remnant will be prograde or retrograde, relative to the rotation of the baryon disk. This code will focus on the angular momentum calculation which will be used to analyze whether the halo is prograde or retrograde.

academic papers used for inspiration and thought:
-------------------------------------------------
Drakos+2019 (discussion of dark matter halo mergers)\
Chua+2019 (claim that with baryon matter, halos are more spherical)\
Teklu+2015 (discussion of angular momentum in disk and halo)\
Carollo+2007 (claim retrograde and prograde MW halo)\
Koppelman+2019 (claim retrograde MW halo)

In [ ]:
# import modules
import numpy as np
import astropy.units as u

# import plotting module
import matplotlib.pyplot as plt

# import previous code
from ReadFile import Read
from CenterOfMass2 import CenterOfMass
from ParticleProperties2 import ParticleProperties

# Rotate M31 Data
The disk of M31 is tilted from the simulation data and we want the data to represent M31 edge on. The following code is from Lab 7.

In [ ]:
def RotateFrame(posI,velI):
    """a function that will rotate the position and velocity vectors
    so that the disk angular momentum is aligned with z axis. 
    
    PARAMETERS
    ----------
        posI : `array of floats`
             3D array of positions (x,y,z)
        velI : `array of floats`
             3D array of velocities (vx,vy,vz)
             
    RETURNS
    -------
        pos: `array of floats`
            rotated 3D array of positions (x,y,z) 
            such that disk is in the XY plane
        vel: `array of floats`
            rotated 3D array of velocities (vx,vy,vz) 
            such that disk angular momentum vector
            is in the +z direction 
    """
    
    # compute the angular momentum
    L = np.sum(np.cross(posI,velI), axis=0)
    
    # normalize the angular momentum vector
    L_norm = L/np.sqrt(np.sum(L**2))

    # Set up rotation matrix to map L_norm to
    # z unit vector (disk in xy-plane)
    
    # z unit vector
    z_norm = np.array([0, 0, 1])
    
    # cross product between L and z
    vv = np.cross(L_norm, z_norm)
    s = np.sqrt(np.sum(vv**2))
    
    # dot product between L and z 
    c = np.dot(L_norm, z_norm)
    
    # rotation matrix
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

    # Rotate coordinate system
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T
    
    return pos, vel

# Angular Momentum
To calculate the angular momentum:
\begin{equation}
    \vec{L} = \sum_{i} \vec{r_i} \times \vec{p_i} = \sum_{i} m_i (\vec{r_i} \times \vec{v_i})
\end{equation}
To answer this question, we need to analyze the direction of the angular momentum of the halo and the angular momentum of the disk. If the dark matter halo is prograde, the orbital rotation is in the same direction and if the dark matter halo is retrograde, the orbital rotation is in the opposite direction. For the variables: $m_i$ is the mass of the individual particle, $r_i$ is the three-dimensional position vector of the particle, adjusted for the center of mass position, and $v_i$ is the three-dimensional velocity vector of the particle, also adjusted for the center of mass velocity. The sum is the addition of the angular momentum for all particles of that type (either dark matter or disk).

In [ ]:
# from lab 7:
# rotate the disk to be edge on for M31

# radius restrictions (homework 5):
# disk = 15 kpc
# halo = 63 kpc

# pick two or three radii to track angular momentum --> pick radii based on shells of mass (1/2 mass of halo and 3/4 mass of halo and...)

In [ ]:
def AngularMomentumMW(type, start, end, n):
    '''
        This function calculates the angular momentum of a galaxy component of the MW

        PARAMETERS:
        -----------
            start: 'int'
                the number of the first snapshot to be read in
            end: 'int'
                the number of the last snapshot to be read in
            n: 'int'
                indicates the interval over while the COM will be returned
            type: 'int' (i.e. 1, 2, or 3)
                the particle type (1 for halo, 2 for disk)

        OUTPUT:
        -------
            AngularMomentumMW: np.array
                the angular momentum of the galaxy component of the MW in kg m^2/s
    '''

    # L = r x p = m * r x v
    # where the variables are coming from:
        # r: from particle properties and COM p
        # v: from particle properties and COM v
        # m: from particle properties
    
    # generating the snapshot id sequence
    snap_ids = np.arange(start, end+n, n)
    # checking that the array is not empty but stopping the code if it is empty
    if snap_ids.size == 0:
        print("no snapshots found (invalid input)")
        return

    # setting tolerance and VolDec for calculating COM_P in CenterOfMass
    delta = 0.1
    volDec = 4.0

    # defining max halo and radii values
    if type == 1:  # halo
        r_max = 62
    elif type == 2:  #disk
        r_max = 15
    else:
        print('bulge type particles not accepted')
        return

    # looping over the txt files within each galaxy folder 
    # each folder is named as the galaxy name (MW, M31, or M33)
    for i, snap_id in enumerate(snap_ids):

        # composing the data filename
        ilbl = f"{snap_id:03d}"  # looks at the last three digits of the snapshot file
        filename = f"MW/MW_{ilbl}.txt" # folder/file

        # assigning variables to the outputs of the Read function
        time, N, data = Read(filename)

        # store variables imported from ParticleProperties
        r, v, m, Npart = ParticleProperties(filename, type, int(N))

        # at any given time, we know com p and com v for the galaxy component (either disk or halo)
        # create an instance of CenterofMass class for galaxy with particle type
        galaxy_COM = CenterOfMass(filename, type)

        #print(f"Calculating COM_P for {filename}: delta={delta}, volDec={volDec}")

        # storing the COM position and COM velocity
        r_COM = galaxy_COM.COM_P(delta, volDec)
        v_COM = galaxy_COM.COM_V(r_COM[0], r_COM[1], r_COM[2])

        # update the radii and velocities to be in the COM frame
        # check shapes --> r_COM will only be one vector
        r_new = r - r_COM.value
        v_new = v - v_COM.value

        r_new_mag = np.zeros(Npart)
        AngularMomentum_ith = np.zeros((Npart,3))
        AngularMomentumMW = np.zeros(3)
        
        for i in range(Npart):
            
            # calculate the distance
            #r_new_mag = np.linalg.norm(r_new)
            sum_of_squares = r_new[i, 0]**2 + r_new[i, 1]**2 + r_new[i, 2]**2
            r_new_mag[i] = np.sqrt(sum_of_squares)
            
            # need the magnitude projected within the plane of the disk?
            # choose along which axis?
            
            if r_new_mag[i] <= r_max:
                
                # calculate angular momentum for the ith particle
                AngularMomentum_ith = m[i] * np.cross(r_new[i], v_new[i])
            
                # total angular momentum for the galaxy component would be the sum of every particle's
                # angular momentum in that galaxy component
                AngularMomentumMW = AngularMomentumMW + AngularMomentum_ith
    
        return AngularMomentumMW


In [ ]:
def AngularMomentumM31(type, start, end, n):
    '''
        This function calculates the angular momentum of a galaxy component of M31

        PARAMETERS:
        -----------
            start: 'int'
                the number of the first snapshot to be read in
            end: 'int'
                the number of the last snapshot to be read in
            n: 'int'
                indicates the interval over while the COM will be returned
            type: 'int' (i.e. 1, 2, or 3)
                the particle type (1 for halo, 2 for disk)

        OUTPUT:
        -------
            AngularMomentumM31: np.array
                the angular momentum of the galaxy component of M31 in kg m^2/s
    '''

    # L = r x p = m * r x v
    # where the variables are coming from:
        # r: from particle properties and COM p
        # v: from particle properties and COM v
        # m: from particle properties
    
    # generating the snapshot id sequence
    snap_ids = np.arange(start, end+n, n)
    # checking that the array is not empty but stopping the code if it is empty
    if snap_ids.size == 0:
        print("no snapshots found (invalid input)")
        return

    # setting tolerance and VolDec for calculating COM_P in CenterOfMass
    delta = 0.1
    volDec = 4.0

    # defining max halo and radii values
    if type == 1:  # halo
        r_max = 62
    elif type == 2:  #disk
        r_max = 20
    else:
        print('bulge type particles not accepted')
        return

    # looping over the txt files within each galaxy folder 
    # each folder is named as the galaxy name (MW, M31, or M33)
    for i, snap_id in enumerate(snap_ids):

        # composing the data filename
        ilbl = f"{snap_id:03d}"  # looks at the last three digits of the snapshot file
        filename = f"M31/M31_{ilbl}.txt" # folder/file

        # assigning variables to the outputs of the Read function
        time, N, data = Read(filename)

        # store variables imported from ParticleProperties
        r, v, m, Npart = ParticleProperties(filename, type, int(N))

        # compute the rotated position and velocity vectors using RotateFrame function
        rn, vn = RotateFrame(r,v)

        # at any given time, we know com p and com v for the galaxy component (either disk or halo)
        # create an instance of CenterofMass class for galaxy with particle type
        galaxy_COM = CenterOfMass(filename, type)

        # storing the COM position and COM velocity
        r_COM = galaxy_COM.COM_P(delta, volDec)
        v_COM = galaxy_COM.COM_V(r_COM[0], r_COM[1], r_COM[2])

        # update the radii and velocities to be in the COM frame
        # check shapes --> r_COM will only be one vector
        r_new = rn - r_COM.value
        v_new = vn - v_COM.value

        r_new_mag = np.zeros(Npart)
        AngularMomentum_ith = np.zeros((Npart,3))
        AngularMomentumM31 = np.zeros(3)
        
        for i in range(Npart):
            
            # calculate the distance
            #r_new_mag = np.linalg.norm(r_new)
            sum_of_squares = r_new[i, 0]**2 + r_new[i, 1]**2 + r_new[i, 2]**2
            r_new_mag[i] = np.sqrt(sum_of_squares)
            
            # need the magnitude projected within the plane of the disk?
            # choose along which axis?
            
            if r_new_mag[i] <= r_max:
                
                # calculate angular momentum for the ith particle
                AngularMomentum_ith = m[i] * np.cross(r_new[i], v_new[i])

                print(f'am ith: {AngularMomentum_ith}')
            
                # total angular momentum for the galaxy component would be the sum of every particle's
                # angular momentum in that galaxy component
                AngularMomentumM31 = AngularMomentumM31 + AngularMomentum_ith
    
        return AngularMomentumM31


In [ ]:
# assign variables to the outputs of AngularMomentum for MW and M31 for their disk and
# halo components and for pre and post merger

# important snapshots: 0, 280 (first interaction), 385 (separated again), 500, 800

# for MW:
'''
# halo, pre-merger:
MW_halo_L = AngularMomentumMW(1, 0, 500, 100)
MW_halo_L_mag = np.linalg.norm(MW_halo_L)

# disk, pre-merger:
MW_disk_L = AngularMomentumMW(2, 0, 500, 100)
MW_disk_L_mag = np.linalg.norm(MW_disk_L)
'''
# for M31:

# halo, pre-merger:
M31_halo_L = AngularMomentumM31(1, 0, 500, 100)
M31_halo_L_mag = np.linalg.norm(M31_halo_L)
'''
# disk, pre-merger:
M31_disk_L = AngularMomentumM31(2, 0, 500, 100)
M31_disk_L_mag = np.linalg.norm(M31_disk_L)


# MW-M31 remnant:

# halo:
remnant_halo_L = AngularMomentumMW(1, 500, 800, 50) + AngularMomentumM31(1, 500, 800, 50)
remnant_halo_L_mag = np.linalg.norm(remnant_halo_L)

# disk:
remnant_disk_L = AngularMomentumMW(2, 500, 800, 50) + AngularMomentumM31(2, 500, 800, 50)
remnant_disk_L_mag = np.linalg.norm(remnant_disk_L)
'''

In [ ]:
# can normalize the angular momentum vector since we only care about direction



# Prograde or Retrograde?
Because the angular momentum vectors are in 3D and the directions are not simply clockwise or counterclockwise, we must calculate the dot product between the two vectors to determine whether they oppose each other or not.
\begin{equation}
    \vec{L_{halo}} \cdot \vec{L_{disk}} = |\vec{L_{halo}}||\vec{L_{disk}}|cos\theta \rightarrow cos\theta = \frac{\vec{L_{halo}} \cdot \vec{L_{disk}}}{|\vec{L_{halo}}||\vec{L_{disk}}|}
\end{equation}
If the cosine term is negative, the orbit of the dark matter halo is prograde. If the cosine term is positive, the orbit of the dark matter halo is retrograde.

In [ ]:
# make function for dot product?
def CosineTerm(vec1, vec2):
    '''
        This function computes the dot pr
    '''

In [ ]:
# compute the dot product of MW components before merger
MW_dot = np.dot(MW_halo_L, MW_disk_L)
# compute the cosine term for MW
MW_cos = MW_dot / (MW_halo_L_mag * MW_disk_L_mag)

# compute the dot product of M31 components before merger
M31_dot = np.dot(M31_halo_L, M31_disk_L)

print(f'M31 halo L: {M31_halo_L}')
print(f'M31 disk L: {M31_disk_L}')
print(f'M31 dot: {M31_dot}')
print(f'M31 halo mag: {M31_halo_L_mag}')
print(f'M31 disk mag: {M31_disk_L_mag}')

# compute the cosine term for M31
M31_cos = M31_dot / (M31_halo_L_mag * M31_disk_L_mag)

# compute the dot product of the remnant components
remnant_dot = np.dot(remnant_halo_L, remnant_disk_L)
# compute the cosine term for the remnant
remnant_cos = remnant_dot / (remnant_halo_L_mag * remnant_disk_L_mag)

In [ ]:
# plot the dot product as a function of time

time =[]

plt.title('The Dot Product for Each Galaxy (MW, M31, or remnant)')

plt.plot(time, MW_cos, lw=5, linestyle=':', color='red', label='MW')
plt.plot(time, M31_cos, lw=6, linestyle='--', color='blue', label='M31')
plt.plot(time, remnant_cos, lw=7, color='violet', label='MW-M31 remnant')

plt.legend()

# plot color bar
cbar = plt.colorbar()
cbar.set_label("", fontsize=15)

# add axis labels
plt.xlabel('time (Myr)', fontsize=22)
plt.ylabel('cosine term', fontsize=22)
'''
#set axis limits
plt.ylim(-10,10)
plt.xlim(-45,45)
'''
# adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# Dependence on Radius
Carollo+2007 claims that the MW halo has both a prograde and retrograde component. How does the angular momentum throughout the MW, M31, and the remnant vary as a function of radius? Does the direction of the angular momentum change within each component?

In [ ]:
# set an array of radii for analysis
# should include the disk radius, halo radius, and one or two radii outside of the halo radius